In [70]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import json
import os
from typing import List, Dict, Union
from pydantic import BaseModel, Field
from langchain.output_parsers import PydanticOutputParser
from openai import OpenAI
from metrics import *
from agent import *
from prompt import *
from scipy.spatial.distance import jensenshannon
from rouge_score import rouge_scorer
from transformers import AutoTokenizer, AutoModel
import torch
from bert_score import score
from transformers import AutoTokenizer, AutoModel
import torch
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

# SOAP Datasets

In [71]:
a_train_df = pd.read_csv("/home/yl3427/cylab/rag_tnm/Task-3_Train.csv") # 603
a_eval_df = pd.read_csv("/home/yl3427/cylab/rag_tnm/Task-3_Eval.csv") # 75
a_test_df = pd.read_csv("/home/yl3427/cylab/rag_tnm/Task-3_Test.csv") # 87

soap_train_df = pd.read_csv("/home/yl3427/cylab/rag_tnm/BioNLP2023-1A-Train.csv") # 765, this includes all of the three above
soap_test_df = pd.read_csv("/home/yl3427/cylab/rag_tnm/BioNLP2023-1A-Test.csv") # for shots

filtered_df = soap_train_df[soap_train_df['File ID'].isin(set(a_test_df['File ID']).union({'190862.txt', '109943.txt', '195790.txt'}))]
len(filtered_df) # 90 - 1 = 89

89

In [7]:
filtered_df.columns

Index(['File ID', 'Assessment', 'Summary', 'Subjective Sections',
       'Objective Sections'],
      dtype='object')

In [ ]:
for i, row in filtered_df.iterrows():
    print(row['Summary'])
    print("-----------")

In [ ]:
len_lst = []
for i in range(len(soap_test_df)):
    row = soap_test_df.iloc[i]
    try:
        length = len(row["Subjectives"]) + len(row["Objectives"]) + len(row["Assessment"]) + len(row["Summary"])
        len_lst.append(length)
    except:
        print(row)

smallest_number = min(len_lst)
print(smallest_number)
index_of_smallest = len_lst.index(smallest_number)
index_of_smallest

In [ ]:
soap_test_df.iloc[50]["Summary"] # ["Objectives"], ["Assessment"], ["Summary"]

In [ ]:
filtered_df.head(2)

# SOAP result

In [27]:
filtered_df['Summary'].iloc[0]

'Coronary Artery Disease; v fibrillation.'

In [73]:
# zero-shot
soa_zs_df = pd.read_csv("/home/yl3427/cylab/rag_tnm/soap_result/1024_soap_soa_zs.csv")
sa_zs_df = pd.read_csv("/home/yl3427/cylab/rag_tnm/soap_result/1024_soap_sa_zs.csv")
so_zs_df = pd.read_csv("/home/yl3427/cylab/rag_tnm/soap_result/1024_soap_so_zs.csv")
a_zs_df = pd.read_csv("/home/yl3427/cylab/rag_tnm/soap_result/1024_soap_a_zs.csv")

# one-shot
soa_os_df = pd.read_csv("/home/yl3427/cylab/rag_tnm/soap_result/1024_soap_soa_os.csv")
sa_os_df = pd.read_csv("/home/yl3427/cylab/rag_tnm/soap_result/1024_soap_sa_os.csv")
so_os_df = pd.read_csv("/home/yl3427/cylab/rag_tnm/soap_result/1024_soap_so_os.csv")
a_os_df = pd.read_csv("/home/yl3427/cylab/rag_tnm/soap_result/1024_soap_a_os.csv")

In [152]:
zs_df_lst = [(soa_zs_df, "soa_zs"), (sa_zs_df, "sa_zs"), (so_zs_df, "so_zs"), (a_zs_df, "a_zs")]
os_df_lst = [(soa_os_df, "soa_os"), (sa_os_df, "sa_os"), (so_os_df, "so_os"), (a_os_df, "a_os")]
ground_truth = filtered_df["Summary"]

a_zs_df.loc[a_zs_df["File ID"].isin({'190862.txt', '109943.txt'})][['File ID','pred']]

a_os_df.loc[a_os_df["File ID"]== '109943.txt']['pred'].values[0]

'ESRD; HD; HTN; HL; acute CHF exacerbation; NSTEMI; DES in LAD; DES in LMCA'

In [143]:
a_os_df.loc[a_os_df["File ID"].isin({'190862.txt', '109943.txt'})][['File ID','pred']]

,File ID,Assessment,Summary,Subjective Sections,Objective Sections,pred
1,109943.txt,"81yo F with h/o ESRD on HD, HTN, HL, acute CHF...",S/P NSTEMI; HTN; Chronic renal failure; HL,"- echo: 75 EF, LVH, PCWP 18\n- dialyzed, overa...",Last dose of Antibiotics:\nInfusions:\nOther I...,ESRD; HD; HTN; HL; acute CHF exacerbation; NST...
28,190862.txt,"81 yo man with CAD s/p CABG, recent NSTEMI at ...",# Cholangitis; CAD; HTN; AoCRF,NaN,WBC\n57.1\n45.1\n32.1\n19.5\n16.0\nHct\n30.1\n...,CAD; CABG; NSTEMI; HTN; CKD; Cholangitis; Hypo...


# SOAP Evaluation

### ROUGE-L F-score

In [159]:
a_zs_df.loc[a_zs_df["File ID"].isin({'190862.txt', '109943.txt'})][['File ID','pred', 'Summary']]

a_zs_df.loc[a_zs_df["File ID"]== '109943.txt']['Summary'].values[0]

'S/P NSTEMI; HTN; Chronic renal failure; HL'

In [160]:
rg_scorer = rouge_scorer.RougeScorer(['rougeL'], use_stemmer=True)

predictions = ['CAD s/p CABG; recent NSTEMI; HTN; CKD; cholangitis; hypotension (resolved)',
               'ESRD on HD; HTN; HL; acute CHF exacerbation; NSTEMI; s/p cath with DES in LAD and LMCA.']

references = ['# Cholangitis; CAD; HTN; AoCRF',
              'S/P NSTEMI; HTN; Chronic renal failure; HL']

for pred, ref in zip(predictions, references):
    scores = rg_scorer.score(ref, pred)
    rougeL_f = scores['rougeL'].fmeasure
    print(f"ROUGE-L F-score: {rougeL_f:.4f}")

ROUGE-L F-score: 0.2667
ROUGE-L F-score: 0.1538


In [154]:
len(so_os_df)

89

### BERTScore - using SapBERT

In [161]:
model_name = "cambridgeltl/SapBERT-from-PubMedBERT-fulltext"

predictions = ['CAD s/p CABG; recent NSTEMI; HTN; CKD; cholangitis; hypotension (resolved)',
               'ESRD on HD; HTN; HL; acute CHF exacerbation; NSTEMI; s/p cath with DES in LAD and LMCA.']

references = ['# Cholangitis; CAD; HTN; AoCRF',
              'S/P NSTEMI; HTN; Chronic renal failure; HL']

P, R, F1 = score(predictions, references, model_type=model_name, lang="en", rescale_with_baseline=True, num_layers=12)

for i in range(len(predictions)):
    print(f"BERTScore F1: {F1[i]:.4f}")

BERTScore F1: 0.6864
BERTScore F1: 0.7081


In [108]:
# Example using BioBERT as a substitute for SapBERT
model_name = "cambridgeltl/SapBERT-from-PubMedBERT-fulltext"

# Define your predictions and references
predictions = sa_zs_df['pred'].tolist()
references = ground_truth.tolist()

# Compute BERTScore
P, R, F1 = score(predictions, references, model_type=model_name, lang="en", rescale_with_baseline=True, num_layers=12)

for i in range(len(predictions)):
    print(f"BERTScore F1: {F1[i]:.4f}")

BERTScore F1: 0.5303
BERTScore F1: 0.7943
BERTScore F1: 0.4721
BERTScore F1: 0.5772
BERTScore F1: 0.6488
BERTScore F1: 0.7530
BERTScore F1: 0.5993
BERTScore F1: 0.8016
BERTScore F1: 0.8617
BERTScore F1: 0.1807
BERTScore F1: 0.5046
BERTScore F1: 0.5235
BERTScore F1: 0.6250
BERTScore F1: 0.6284
BERTScore F1: 0.5627
BERTScore F1: 0.8303
BERTScore F1: 0.6942
BERTScore F1: 0.6233
BERTScore F1: 0.5613
BERTScore F1: 0.5967
BERTScore F1: 0.6888
BERTScore F1: 0.6534
BERTScore F1: 0.7050
BERTScore F1: 0.7630
BERTScore F1: 0.8544
BERTScore F1: 0.8415
BERTScore F1: 0.8873
BERTScore F1: 0.7681
BERTScore F1: 0.6788
BERTScore F1: 0.6820
BERTScore F1: 0.4442
BERTScore F1: 0.6995
BERTScore F1: 0.6105
BERTScore F1: 0.7246
BERTScore F1: 0.8055
BERTScore F1: 0.7848
BERTScore F1: 0.5495
BERTScore F1: 0.5635
BERTScore F1: 0.6460
BERTScore F1: 0.7968
BERTScore F1: 0.7658
BERTScore F1: 0.8370
BERTScore F1: 0.6432
BERTScore F1: 0.5750
BERTScore F1: 0.5535
BERTScore F1: 0.6493
BERTScore F1: 0.5766
BERTScore F1:

### Sentence Embedding (the last hidden layer)

In [66]:
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)

In [67]:
device = torch.device('cuda:5' if torch.cuda.is_available() else 'cpu')
model.to(device)

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(30522, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0-11): 12 x BertLayer(
        (attention): BertAttention(
          (self): BertSdpaSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False

In [68]:
def get_sentence_embedding(text):
    inputs = tokenizer(text, return_tensors='pt', truncation=True, max_length=1024) # , truncation=True, max_length=1024
    inputs = {k: v.to(device) for k, v in inputs.items()}
    with torch.no_grad():
        outputs = model(**inputs)
    # Take the mean of the last hidden states
    embeddings = outputs.last_hidden_state.mean(dim=1).cpu().numpy()
    return embeddings

In [162]:
predictions = ['CAD s/p CABG; recent NSTEMI; HTN; CKD; cholangitis; hypotension (resolved)',
               'ESRD on HD; HTN; HL; acute CHF exacerbation; NSTEMI; s/p cath with DES in LAD and LMCA.']

references = ['# Cholangitis; CAD; HTN; AoCRF',
              'S/P NSTEMI; HTN; Chronic renal failure; HL']


pred_embeddings = [get_sentence_embedding(pred) for pred in predictions]
ref_embeddings = [get_sentence_embedding(ref) for ref in references]

for pred_emb, ref_emb in zip(pred_embeddings, ref_embeddings):
    cos_sim = cosine_similarity(pred_emb, ref_emb)[0][0]
    print(f"Sentence Embedding Cosine Similarity: {cos_sim:.4f}")

Sentence Embedding Cosine Similarity: 0.6855
Sentence Embedding Cosine Similarity: 0.7301


In [98]:
pred_embeddings[0][0]

array([-1.27473426e+00,  1.14982224e+00,  1.59178317e-01, -1.58149134e-02,
       -3.00850756e-02,  8.38229895e-01, -7.47969598e-02,  9.24611315e-02,
        1.56802416e-01, -1.09584615e-01, -8.18211436e-02,  3.61887783e-01,
       -3.33906114e-01,  1.77034795e-01, -4.07884240e-01,  5.33148885e-01,
        1.25509143e-01,  2.76233792e-01, -7.53015280e-01,  2.70296991e-01,
        1.07255197e+00,  9.00899529e-01, -2.46315569e-01, -3.50704432e-01,
       -3.49676400e-01,  5.57414055e-01, -1.10942960e-01,  7.61503160e-01,
        1.09017208e-01, -9.14196372e-01, -5.83698690e-01, -3.62480849e-01,
        3.00618291e-01,  8.13169599e-01,  2.96523660e-01, -3.95457149e-01,
       -1.13996472e-02, -3.04619759e-01,  2.18756497e-01,  8.19626033e-01,
        7.56906390e-01, -5.95983386e-01, -3.27615738e-01,  2.78213084e-01,
       -2.89360493e-01, -7.85137296e-01, -3.29415761e-02, -4.69540775e-01,
       -1.73708394e-01,  4.00951505e-01, -3.89522731e-01, -1.21252999e-01,
       -5.64650536e-01,  

In [99]:
cosine_similarity(pred_embeddings[0], ref_embeddings[0])

array([[0.76647013]], dtype=float32)

### ALL

In [ ]:
rg_scorer = rouge_scorer.RougeScorer(['rougeL'], use_stemmer=True)
model_name = "cambridgeltl/SapBERT-from-PubMedBERT-fulltext"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)
device = torch.device('cuda:5' if torch.cuda.is_available() else 'cpu')
model.to(device)

In [ ]:
# def get_sentence_embedding(text):
#     inputs = tokenizer(text, return_tensors='pt', truncation=True, max_length=1024) # , truncation=True, max_length=1024
#     inputs = {k: v.to(device) for k, v in inputs.items()}
#     with torch.no_grad():
#         outputs = model(**inputs)
#     # Take the mean of the last hidden states
#     embeddings = outputs.last_hidden_state.max(dim=1).values.cpu().numpy()
#     return embeddings
def get_sentence_embedding(text):
    inputs = tokenizer(text, return_tensors='pt', truncation=True, max_length=1024)
    inputs = {k: v.to(device) for k, v in inputs.items()}
    with torch.no_grad():
        outputs = model(**inputs)
    # Mean pooling
    mean_embedding = outputs.last_hidden_state.mean(dim=1)
    # Max pooling
    max_embedding = outputs.last_hidden_state.max(dim=1).values
    # Concatenate mean and max
    combined_embedding = torch.cat((mean_embedding, max_embedding), dim=1).cpu().numpy()
    return max_embedding.cpu().numpy()

In [ ]:
zs_df_lst = [(soa_zs_df, "soa_zs"), (sa_zs_df, "sa_zs"), (so_zs_df, "so_zs"), (a_zs_df, "a_zs")]
os_df_lst = [(soa_os_df, "soa_os"), (sa_os_df, "sa_os"), (so_os_df, "so_os"), (a_os_df, "a_os")]
ground_truth = filtered_df["Summary"]

zs_outer_dict = {}
os_outer_dict = {}

for zs_df, zs_name in zs_df_lst:
    print(zs_name)
    zs_outer_dict[zs_name] = {'rougeL': {'scores':[]}, 'bert_score': {'scores':[]}, 'sent_emb_sim': {'scores':[]}}
    for pred, ref in zip(zs_df['pred'].tolist(), ground_truth.tolist()):

        # Calculate ROUGE-L F-score
        scores = rg_scorer.score(ref, pred)
        rougeL_f = scores['rougeL'].fmeasure
        zs_outer_dict[zs_name]['rougeL']['scores'].append(rougeL_f)

        # Calculate embedding similarity
        pred_emb = get_sentence_embedding(pred)
        ref_emb = get_sentence_embedding(ref)
        sim = cosine_similarity(pred_emb, ref_emb)[0][0]
        zs_outer_dict[zs_name]['sent_emb_sim']['scores'].append(sim)

    # Calculate BERTScore
    _, _, bert_scores = score(zs_df['pred'].tolist(), ground_truth.tolist(), model_type=model_name, lang="en", rescale_with_baseline=True, num_layers=12)
    zs_outer_dict[zs_name]['bert_score']['scores'] = bert_scores.tolist()

    zs_outer_dict[zs_name]['rougeL']['average'] = np.mean(zs_outer_dict[zs_name]['rougeL']['scores'])
    zs_outer_dict[zs_name]['rougeL']['std'] = np.std(zs_outer_dict[zs_name]['rougeL']['scores'])

    zs_outer_dict[zs_name]['bert_score']['average'] = np.mean(zs_outer_dict[zs_name]['bert_score']['scores'])
    zs_outer_dict[zs_name]['bert_score']['std'] = np.std(zs_outer_dict[zs_name]['bert_score']['scores'])
    
    zs_outer_dict[zs_name]['sent_emb_sim']['average'] = np.mean(zs_outer_dict[zs_name]['sent_emb_sim']['scores'])
    zs_outer_dict[zs_name]['sent_emb_sim']['std'] = np.std(zs_outer_dict[zs_name]['sent_emb_sim']['scores'])

for os_df, os_name in os_df_lst:
    print(os_name)
    os_outer_dict[os_name] = {'rougeL': {'scores':[]}, 'bert_score': {'scores':[]}, 'sent_emb_sim': {'scores':[]}}
    for pred, ref in zip(os_df['pred'].tolist(), ground_truth.tolist()):

        # Calculate ROUGE-L F-score
        scores = rg_scorer.score(ref, pred)
        rougeL_f = scores['rougeL'].fmeasure
        os_outer_dict[os_name]['rougeL']['scores'].append(rougeL_f)

        # Calculate embedding similarity
        pred_emb = get_sentence_embedding(pred)
        ref_emb = get_sentence_embedding(ref)
        sim = cosine_similarity(pred_emb, ref_emb)[0][0]
        os_outer_dict[os_name]['sent_emb_sim']['scores'].append(sim)

    # Calculate BERTScore
    _, _, bert_scores = score(os_df['pred'].tolist(), ground_truth.tolist(), model_type=model_name, lang="en", rescale_with_baseline=True, num_layers=12)
    os_outer_dict[os_name]['bert_score']['scores'] = bert_scores.tolist()

    os_outer_dict[os_name]['rougeL']['average'] = np.mean(os_outer_dict[os_name]['rougeL']['scores'])
    os_outer_dict[os_name]['rougeL']['std'] = np.std(os_outer_dict[os_name]['rougeL']['scores'])

    os_outer_dict[os_name]['bert_score']['average'] = np.mean(os_outer_dict[os_name]['bert_score']['scores'])
    os_outer_dict[os_name]['bert_score']['std'] = np.std(os_outer_dict[os_name]['bert_score']['scores'])
    
    os_outer_dict[os_name]['sent_emb_sim']['average'] = np.mean(os_outer_dict[os_name]['sent_emb_sim']['scores'])
    os_outer_dict[os_name]['sent_emb_sim']['std'] = np.std(os_outer_dict[os_name]['sent_emb_sim']['scores'])

soa_zs


sa_zs


so_zs


a_zs


soa_os


sa_os


so_os


a_os


In [ ]:
#  Max Pooling
for method in zs_outer_dict.keys():
    print(f"{method} ROUGE-L: {zs_outer_dict[method]['rougeL']['average'] * 100:.2f} ± {zs_outer_dict[method]['rougeL']['std'] * 100:.2f}")
    print(f"{method} BERTScore: {zs_outer_dict[method]['bert_score']['average'] * 100:.2f} ± {zs_outer_dict[method]['bert_score']['std'] * 100:.2f}")
    print(f"{method} Sentence Embedding Similarity: {zs_outer_dict[method]['sent_emb_sim']['average'] * 100:.2f} ± {zs_outer_dict[method]['sent_emb_sim']['std'] * 100:.2f}")
    print("-----------")

soa_zs ROUGE-L: 19.27 ± 12.61
soa_zs BERTScore: 66.98 ± 11.80
soa_zs Sentence Embedding Similarity: 68.60 ± 12.28
-----------
sa_zs ROUGE-L: 20.83 ± 12.59
sa_zs BERTScore: 67.62 ± 12.00
sa_zs Sentence Embedding Similarity: 68.99 ± 12.47
-----------
so_zs ROUGE-L: 7.47 ± 8.42
so_zs BERTScore: 53.58 ± 12.50
so_zs Sentence Embedding Similarity: 56.56 ± 13.57
-----------
a_zs ROUGE-L: 22.35 ± 13.62
a_zs BERTScore: 68.52 ± 12.94
a_zs Sentence Embedding Similarity: 69.60 ± 13.34
-----------


In [ ]:
for method in os_outer_dict.keys():
    print(f"{method} ROUGE-L: {os_outer_dict[method]['rougeL']['average'] * 100:.2f} ± {os_outer_dict[method]['rougeL']['std'] * 100:.2f}")
    print(f"{method} BERTScore: {os_outer_dict[method]['bert_score']['average'] * 100:.2f} ± {os_outer_dict[method]['bert_score']['std'] * 100:.2f}")
    print(f"{method} Sentence Embedding Similarity: {os_outer_dict[method]['sent_emb_sim']['average'] * 100:.2f} ± {os_outer_dict[method]['sent_emb_sim']['std'] * 100:.2f}")
    print("-----------")

soa_os ROUGE-L: 22.64 ± 13.89
soa_os BERTScore: 68.49 ± 11.90
soa_os Sentence Embedding Similarity: 68.91 ± 12.22
-----------
sa_os ROUGE-L: 23.93 ± 13.74
sa_os BERTScore: 68.14 ± 11.72
sa_os Sentence Embedding Similarity: 68.38 ± 12.10
-----------
so_os ROUGE-L: 8.23 ± 9.96
so_os BERTScore: 52.74 ± 13.23
so_os Sentence Embedding Similarity: 53.68 ± 13.84
-----------
a_os ROUGE-L: 25.99 ± 15.55
a_os BERTScore: 68.61 ± 13.57
a_os Sentence Embedding Similarity: 68.68 ± 14.10
-----------


In [ ]:
# 마지막: Max Pooling
for method in zs_outer_dict.keys():
    print(method)
    print(zs_outer_dict[method]['rougeL']['average'])
    # print(zs_outer_dict[method]['rougeL']['std'])
    print(zs_outer_dict[method]['bert_score']['average'])
    # print(zs_outer_dict[method]['bert_score']['std'])
    print(zs_outer_dict[method]['sent_emb_sim']['average'])
    # print(zs_outer_dict[method]['sent_emb_sim']['std'])
    print("-----------")

soa_zs
0.1927351545038894
0.6697580941607443
0.68596196
-----------
sa_zs
0.20829924435646152
0.6762044983968306
0.68985057
-----------
so_zs
0.07471598111256489
0.5358337954189001
0.565636
-----------
a_zs
0.22350889743008542
0.6852228117457936
0.6959635
-----------


In [ ]:
# 마지막: Mean Pooling
for method in zs_outer_dict.keys():
    print(method)
    print(zs_outer_dict[method]['rougeL']['average'])
    # print(zs_outer_dict[method]['rougeL']['std'])
    print(zs_outer_dict[method]['bert_score']['average'])
    # print(zs_outer_dict[method]['bert_score']['std'])
    print(zs_outer_dict[method]['sent_emb_sim']['average'])
    # print(zs_outer_dict[method]['sent_emb_sim']['std'])
    print("-----------")

soa_zs
0.1927351545038894
0.6697580941607443
0.66190165
-----------
sa_zs
0.20829924435646152
0.6762044983968306
0.6685913
-----------
so_zs
0.07471598111256489
0.5358337954189001
0.52832097
-----------
a_zs
0.22350889743008542
0.6852228117457936
0.6769677
-----------
